# Visualizing 3D data in napari
In this tutorial, you will learn:
- How to load images from hdf5 files
- How to open images in napari
- The difference between intensity images and label images
- How to visualize measurements on images using a napari plugin
- How to explore image data interactively

In [2]:
import h5py
# We import some custom functions to handle the hdf5 files
from h5_files import h5_select, h5_summary, to_numpy

In [3]:
# Get an overview of what's in an hdf5 file
file_path = r'bio325_material\20210416_Timecourse5EU_s5_NA.h5'
print(h5_summary(file_path))

filename:	20210416_Timecourse5EU_s5_NA.h5
condition:	ZE injected with 5-EU 2.5 - 5hpf, MeOH permeabilized, co-stained with SYTOX, bCatenin and PCNA

       name             shape   dtype     size     element_size_um  img_type    stain cycle wavelength level
    ch_00/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity     5-EU     0        647     0
    ch_00/1 (286, 1024, 1024)  uint16   599.79   [1.0, 0.65, 0.65] intensity     5-EU     0        647     1
    ch_00/2   (286, 512, 512)  uint16   149.95     [1.0, 1.3, 1.3] intensity     5-EU     0        647     2
    ch_01/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity bCatenin     0        568     0
    ch_01/1 (286, 1024, 1024)  uint16   599.79   [1.0, 0.65, 0.65] intensity bCatenin     0        568     1
    ch_01/2   (286, 512, 512)  uint16   149.95     [1.0, 1.3, 1.3] intensity bCatenin     0        568     2
    ch_02/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity     PCNA   

In [7]:
# Load in the images from the hd5f file
f = h5py.File(file_path, "r")
sytox = h5_select(f, {'stain': 'SYTOX', 'level': 0})[0]
sytox_np = to_numpy(sytox)

In [45]:
# TODO: Check what we loaded with the h5_select function
print(sytox)
print(sytox.shape)  # numpy axis order of shape() = (z, x, y)
print(sytox_np[0:2])

<HDF5 dataset "0": shape (286, 2048, 2048), type "<u2">
(286, 2048, 2048)
[[[ 5  0  2 ...  2 10  7]
  [ 0  6  5 ...  0 10  5]
  [ 0  6  0 ...  3  9  0]
  ...
  [ 0  0  0 ... 19 16 21]
  [ 0  0 20 ...  0  9 14]
  [ 0  2 16 ...  0  0 12]]

 [[ 4  0  0 ...  7  0  8]
  [ 2  3  0 ...  6  6  0]
  [ 1  3  4 ...  5  3  5]
  ...
  [ 2  4  2 ...  9  0  2]
  [ 6 22 10 ...  4 14  0]
  [ 8 16  2 ... 12 12  2]]]


In [48]:
# TODO: Load the PCNA, the bCatenin and the 5-EU images at level 0
print(h5_select(f, {'stain': 'PCNA', 'level': 0})[0])
PCNA = h5_select(f, {'stain': 'PCNA', 'level': 0})[0]
PCNA_np = to_numpy(PCNA)

bCatenin = h5_select(f, {'stain': 'bCatenin', 'level': 0})[0]
bCatenin_np = to_numpy(bCatenin)

zyg_5EU = h5_select(f, {'stain': '5-EU', 'level': 0})[0]
zyg_5EU_np = to_numpy(zyg_5EU)

<HDF5 dataset "0": shape (286, 2048, 2048), type "<u2">


In [49]:
# Loading the segmentation mask for the nuclei
nuclei = h5_select(f, {'stain': 'nuclei', 'level': 2})[0]
    # stain: nuclei has shape (286, 512, 512), does/could this create problems with plotting (different dimensions)?
nuclei_np = to_numpy(nuclei)

In [50]:
# TODO: Load the segmentation masks for the cells & the membranes

# loading segm. masks for cells
cells = h5_select(f, {'stain': 'cells', 'level': 2})[0]
cells_np = to_numpy(cells)

# loading segm. masks for membranes
membrane = h5_select(f, {'stain': 'membrane', 'level': 2})[0]
membrane_np = to_numpy(membrane)

In [51]:
# Each dataset contains the actual data plus some metadata attributes
# As you can see here, the nuclear segmentation was done using cellpose: https://www.cellpose.org 
list(nuclei.attrs)

['cellpose_cellprob_th',
 'cellpose_d',
 'element_size_um',
 'img_type',
 'level',
 'stain']

## The napari viewer
napari is a fast, interactive, multi-dimensional image viewer for Python. It’s designed for browsing, annotating, and analyzing large multi-dimensional images.  
https://napari.org

In [54]:
import napari

In [61]:
# Open the image and the segmentation in the napari viewer
viewer = napari.view_image(sytox_np, scale=sytox.attrs['element_size_um'])
viewer.add_labels(nuclei_np, scale=nuclei.attrs['element_size_um'])
    # RuntimeError: Cannot run the event loop while another loop is running
    # solution: I already had napari opened (in a separate window) - anyhow

<Labels layer 'nuclei_np' at 0x23648fa57c0>

Exception in callback BaseAsyncIOLoop._handle_events(7092, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(7092, 1)>
Traceback (most recent call last):
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\popsicle cell\.conda\envs\bio325_2021\

In [62]:
# TODO: Add other intensity images & segmentations to the viewer & explore
viewer.add_image(PCNA_np, scale=PCNA.attrs['element_size_um'])
viewer.add_image(bCatenin_np, scale=bCatenin.attrs['element_size_um'])
viewer.add_image(zyg_5EU_np, scale=zyg_5EU.attrs['element_size_um'])
viewer.add_labels(cells_np, scale=cells.attrs['element_size_um'])
viewer.add_labels(membrane_np, scale=membrane.attrs['element_size_um'])
# Make sure to add the membrane segmentation to see the cell boundaries

C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\napari\utils\events\event.py:781: RuntimeWarning: coroutine 'get_msg' was never awaited
  self.add(**emitters)  # type: ignore


<Labels layer 'membrane_np' at 0x2363517e580>

C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\magicgui\events.py:40: FutureWarning: 

magicgui 0.4.0 will change the way that callbacks are called.
Instead of a single `Event` instance, with an `event.value` attribute,
callbacks will receive the value(s) directly:

@DataFrame.changed.connect
def my_callback(*args):
    # *args are the value(s) themselves!

To silence this warning you may either provide a callback that has more
or less than 1 parameter.  Or annotate the single parameter as anything
*other* than `Event`, e.g. `def callback(x: int): ...`
For details, see: https://github.com/napari/magicgui/issues/255
  warnings.warn(
C:\Users\popsicle cell\.conda\envs\bio325_2021\lib\site-packages\magicgui\events.py:40: FutureWarning: 

magicgui 0.4.0 will change the way that callbacks are called.
Instead of a single `Event` instance, with an `event.value` attribute,
callbacks will receive the value(s) directly:

@feature.changed.connect
def my_callback(*args):
    # *a

# Questions to look at during the exploration
- How does nuclear size & cell size vary over the embryo?
- How does the 5-EU signal vary over the different nuclei and what does that mean?
- How does PCNA vary over the embryo and what does that mean?
- What would you need to measure to quantitatively answer those questions?
- What potential technical biases do you see in the images that we should be aware of when quantifying them?

In [ ]:
# Hint: in napari: click tab 'Plugins > napari-feature-visualization > feature_vis' to e.g. colourmap labels to certain features.


## Visualize feature measurements
Using the napari feature visualization plugin, look at some of the features, e.g.:  
- Physical Size of the nuclei or the cell. Roundness
- EU intensity (mean, skewness)
- PCNA intensities (mean)  
Do the quantifications help you answer the questions above?